In [17]:
import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
from sklearn.preprocessing import MinMaxScaler

## Shell Algorithm

In [18]:
def shell(seq): # N log N ou N^6/5 
  inc = len(seq) // 2
  while inc:
    for i, el in enumerate(seq[inc:], inc):
      while i >= inc and seq[i - inc] > el:
        seq[i] = seq[i - inc]
        i -= inc
      seq[i] = el
    inc = 1 if inc == 2 else inc * 5 // 11

## Frequency Count Method

In [19]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(shell)

  # create a list to Shell with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [20]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout



[ Info: Started!
0.0%┣                                             ┫ 0/600 [00:00<00:-29, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           2.534e+05  1.594e+01  y = x₀                                        3           2.248e+05  6.000e-02  y = (x₀ + x₀)                                 4           1.819e+05  2.118e-01  y = (x₀ * log(x₀))                            5           2.240e+04  2.094e+00  y = (square(log(x₀)) * x₀)                    8           1.303e+04  1.808e-01  y = ((x₀ + log(x₀)) * square(log(x₀)))        11          5.024e+03  

In [21]:
registros_ = registros

In [22]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

print(lista_melhor_valor[4])

4.9809117*x0*log(x0)
